# Genetic Algorithm 
## Coloquio de Matemáticas, ITAM (2023). 

@author: Diego Velázquez Trejo

@github: DiegoArturoVelazquezTrejo

@linkedin: linkedin.com/in/diego-velazquez-32a4b8244

### Required libraries

In [ ]:
# Library for matrix operations
import numpy as np 
# Library for time operations
import time 
# Library for random operations
import random 
# Library for plotting
from matplotlib import pyplot as plt 


# Auxiliar functions 

In [ ]:
# Function that generates a random letter
def new_Character():
    return random.choice('abcdefghijklmnopqrstuvwxyz ')

In [ ]:
# Function that prints the population 
def print_pop(population): 
  print("Population: ")
  for i in range(len(population)): 
    if(i % 10 == 0): 
      print("\n")
    print(population[i], end = ", ")


# Genetic algo

In [ ]:
'''
Genetic algorithm
'''
class GeneticWordAlgo: 

    # Constructor of the class  
    def __init__(self, initialPop, goal, cross_rate = 0.3, mutation_rate = 0.3, method = "cross.over",  gens = 100): 
        # Population
        self.population = initialPop
        # Cross rate 
        self.cross_rate = cross_rate
        # Mutation rate 
        self.mutation_rate = mutation_rate
        # Cross Method 
        self.method = method 
        # Number of cicles to be done 
        self.gens = gens
        # Array that contains the fitness measures 
        self.fitness_measures = np.zeros(len(self.population))
        # Algorithm's goal 
        self.goal = goal

    # Objetive function 
    def wordFun(self, word): 
        fitness = 0 
        for i in range(0,len(word)):
            if(word[i] == self.goal[i]):
                fitness = fitness + 1
        return fitness

    # Crossover methods 
    def crossOver(self, word1, word2):
        # We get the middle point of the word 
        middle = int(len(word1)/2)
        adn1 = word1[0:middle]
        adn2 = word2[middle:len(word2)]
        new_word = adn1 + adn2
        return new_word

    # Crossover Toss 
    def crossOverToss(self, word1, word2):
        word = word1
        for i in range(0, len(word1)):
            if(random.random() < self.cross_rate):
                word = word.replace(word[i], word2[i])
        return word
    
    # Probabilistic cross over
    def cross_over_probabilistic(self, word1, word2):
        num = int(random.random() * (len(word1) - 1))
        adn1 = word1[:num]
        adn2 = word2[num:]
        return adn1 + adn2

    # Mutation methods 
    def mutate_word(self,word1):
        son = word1
        for i in range(0, len(word1)):
            if(random.random() < self.mutation_rate):
                son = son.replace(son[i], new_Character())
        return son

    # Selection methods
    def select_cross_method(self, word1, word2):
        funs = {'cross.over': self.crossOver, 
                'cross.over.probabilistic': self.cross_over_probabilistic, 
                'cross.over.toss': self.crossOverToss}
        return funs[self.method](word1, word2)
        

    # Fitness function (returns probabilities of each individual)
    def fitness(self):
        for i in range(0, len(self.population)):
            self.fitness_measures[i] = self.wordFun(self.population[i])
        # Calculate probabilities of each individual. We will standarize this vector using min max value 
        self.fitness_measures = (self.fitness_measures-np.min(self.fitness_measures))/(np.max(self.fitness_measures)-np.min(self.fitness_measures))

    # Bernoui's method (selection)
    def bernoulli(self):
        words = []; i = 0
        while(i != 2):
            # Select the index of a random word in the population
            index_word = int(random.random() * len(self.population))
            # Select a random number between 0 and 1
            if( random.random() < self.fitness_measures[index_word]):
                words.append(self.population[index_word]); i += 1 

        return words[0], words[1]

    # Algorithm's main function
    def run(self):
        print_pop(self.population)
        print("Goal: ", self.goal)
        print("Cross rate: ", self.cross_rate)
        print("Mutation rate: ", self.mutation_rate)
        print("Cross method: ", self.method)
        # Array with the mean for each gen 
        fit = []
        # Iterate for each generation
        t0 = time.time()
        for i in range(self.gens): 
            # Calculate the prob for each individual
            self.fitness()
            # Iterate for each individual in the population
            new_pop = []
            for j in range(len(self.population)):
                # Get the parents for cross over
                vater, mutter = self.bernoulli()
                sohn = self.select_cross_method(vater, mutter)
                new_pop.append(self.mutate_word(sohn))
            
            # Get the best individual 
            max_i = np.argmax(self.fitness_measures)
            best_word = self.population[max_i]
            self.population = new_pop
            self.population[-1] = best_word

            # Report results
            if(i % 20 == 0): 
                print("------------------------------------------------------")
                print("| Generation: ", i, " Average pop: ", np.mean(self.fitness_measures),"|")
                print("------------------------------------------------------")
            fit.append(np.mean(self.fitness_measures))
        
        print(f"Total time {time.time() - t0} secs")
        return self.population, best_word, fit


Lets play!

In [ ]:

# Generate the initial population 
initialPop = []
N = 200
goal = "eduardo"
gens = 100

for i in range(N): 
    word = ""
    for j in range(len(goal)):
        word = word + new_Character()
    initialPop.append(word)



In [ ]:

# Create the genetic algorithm
genetic = GeneticWordAlgo(initialPop, goal, cross_rate = 0.3, mutation_rate = 0.01, method = "cross.over.probabilistic", gens = gens)
res, best, fit = genetic.run()


# Now, we get the best element and the final population

In [ ]:
print(f"Final population: ")
print_pop(res)
print("\n\n\n---------------------------------------------\n\n")
print(f"The element that maximizes the objetive function is: \t\t{best}")

Plotting results

In [ ]:
# Plotting the results
plt.xlabel('Generations')
plt.ylabel('fit value')
plt.plot(fit, c = 'red')
plt.title('Fit through gens')
plt.show()